# Part 1: Cross validation

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from ISLP import load_data
from ISLP.models import (ModelSpec as MS,
                         summarize,
                         poly)
from sklearn.model_selection import train_test_split
from functools import partial
from sklearn.model_selection import \
     (cross_validate,
      KFold,
      ShuffleSplit)
from sklearn.base import clone
from ISLP.models import sklearn_sm

In [3]:
import warnings
warnings.filterwarnings('ignore')

In this part we follow the first part of the lab of Section 4 of our textbook to learn how to implement cross validation in Python (see [here](https://islp.readthedocs.io/en/latest/labs/Ch05-resample-lab.html) for the original version of the Lab)


## 1a) Validation set approach

**Objective of part 1a)**: Use the validation set approach to evaluate the performance of a model predicting `mpg` in the `Auto` dataset based on predictor `horsepower`.

In [ ]:
# Run this cell to load the dataset
Auto =  load_data('Auto')

### Task 1.1: 
Split the dataset into a train and a test part. Your test set should have 196 samples.

In [ ]:
train, validation = train_test_split(Auto, test_size=196)

In [ ]:
train

In [ ]:
validation

### Task 1.2:
Train a simple linear regression model on the training set which predicts `mpg` based on the unique predictor `horsepower`. Make sure to use the training set to train your model.

In [ ]:
design = MS(['horsepower']).fit(train)
X_train = design.transform(train)
X_train

In [1]:
y_train = train.mpg

In [ ]:
model = sm.OLS(y_train,X_train)
results = model.fit()
summarize(results)

### Task 1.3: 
Now use the validation set to compute an estimate for the model's MSE (mean squared error).

In [ ]:
y_valid_actual = validation.mpg
X_validation = design.transform(validation)
y_valid_predicted = results.predict(X_validation)
MSE = np.mean((y_valid_actual - y_valid_predicted)**2)
MSE

### Task 1.4:
Below you find a preimplemented function `evalMSE()`. Explain what this function does (what are the arguments, what is the output)?

In [ ]:
def evalMSE(predictors,
           train,
           validation):
    # build design matrix and response vector
    design = MS(predictors).fit(train)
    X_train = design.transform(train)
    y_train = train.mpg 

    # train model
    model = sm.OLS(y_train,X_train)
    results = model.fit()

    # compute MSE
    y_valid_actual = validation.mpg
    X_validation = design.transform(validation)
    y_valid_predicted = results.predict(X_validation)
    MSE = np.mean((y_valid_actual - y_valid_predicted)**2)
    return MSE

In [ ]:
evalMSE(['horsepower'], train, validation)

*Solution*: The function takes a set of predictors, a dataframe of training data and a dataframe of validation data as inputs. Based on these it trains a linear regression model on the training data using the predictors provided as arguments. It then computes the test MSE on the provided validation set. This test MSE is then returned as output.

### Task 1.5:

Use the function `evalMSE()` to estimate the MSE on the validation set for linear regression models including successively higher polynomial terms of `horsepower` ranging from degree 1 to degree 3.

In [ ]:
from ISLP.models import poly
MSE = []
for i in range(1,4):
    predictors = [poly('horsepower', i+1)] # choose powers of horsepower as predictors up to degree i+1
    err = evalMSE(predictors, train, validation)
    MSE.append(err)

In [ ]:
MSE

## 1b) Cross validation

Cross validation is implemented most comfortably in `scikit-learn`. In order to use the `scikit-learn` implementation of cross validation with our `statsmodels` linear model, we use the wrapper `sklearn_sm` provided by the `ISLP` library. From the lab in Chapter 5:

"The class `sklearn_sm()` has as its first argument a model from `statsmodels`. It can take two additional optional arguments: `model_str` which can be used to specify a formula, and `model_args` which should be a dictionary of additional arguments used when fitting the model. For example, to fit a logistic regression model we have to specify a family argument. This is passed as `model_args={'family':sm.families.Binomial()}`."

After specifying our design matrix `X` and the vector `y` we call the `scikit-learn` function `cross_validate` (see [documentation](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html)).

The result is a dictionary which among others contains the `test_score` which we are interested when we use cross validation to estimate the test error.

### Task 1.6:
Run the following cell to initialize an `sklearn_sm` object with a `statsmodels.OLS` model and a `ModelSpec` object with specified predictor `horsepower`, with dataset `X` with removed `mpg` column und with response vector `y`.

In [ ]:
model = sklearn_sm(sm.OLS,
                  MS(['horsepower']))
X = Auto.drop(columns = ['mpg'])
y = Auto['mpg']

The function `cross_validate` is the original `scikit-learn` function which carries out cross validation. We provide the following arguments:
- a model which needs `fit()` and `predict()` methods
- a design matrix `X` and a vector of training labels `y`
- the parameter `cv` specifying the number of folds for cross validation.

See the [official documentation](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html) for more details.

### Task 1.7:
Run a Leave-one-out-cross-validation for a simple linear regression model predicting `mpg` based on `horsepower`. To do so, choose as many folds as your dataset has samples using the parameter `cv`.

In [ ]:
cv_results = cross_validate(model,
                           X,
                           y,
                           cv = Auto.shape[0])

In [ ]:
cv_err = np.mean(cv_results['test_score'])
cv_err

### Task 1.8:

We can repeat this procedure to compare different models. In the following we do this with various polynomial fits.

Compute the LOOCV error for models to predict `mpg` based on polynomial powers of `mpg` for degrees 1 to 5.

In [ ]:
cv_error = np.zeros(5) #initialize empty vector for CV errors of 5 models
M = sklearn_sm(sm.OLS)
y = Auto['mpg']

# loop over 5 different polynomial models
for i in range(1,6):
    # choose power of horsepowers as predictors up to degree i+1
    predictors = [poly('horsepower', i+1)]
    X = MS(predictors).fit_transform(Auto)
    #perform cross validation for polynomial model of degree i+i
    cv_results = cross_validate(M,
                               X,
                               y,
                               cv = Auto.shape[0])
    # save cross validation error into result vector
    cv_error[i] = np.mean(cv_results['test_score'])
cv_error

### Task 1.9:

Instead of using $K = n$ folds such as above (resulting in Leave-One-Out-Cross-Validation (LOOCV)) we can also specify a smaller integer $K$ of folds. There are two possibilities for this:
- set `cv = K`,
- specify a cross validation generator such as `KFold` (see [documentation](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html))

It is recommended to use the second approach, which is generally preferred as we do better control the kind of split when using `KFold`.

Compute the cross validation error for models to predict `mpg` based on polynomial powers of `mpg` for degrees 1 to 5. Use 10 folds.

In [ ]:
cv_error = np.zeros(5)

cross_val = KFold(n_splits = 10,
                 shuffle = True,
                 random_state = 42)

M = sklearn_sm(sm.OLS)
y = Auto['mpg']

# loop over 5 different polynomial models
for i in range(5):
    # choose power of horsepowers as predictors up to degree i+1
    predictors = [poly('horsepower', i+1)]
    X = MS(predictors).fit_transform(Auto)
    #perform cross validation for polynomial model of degree i+i
    cv_results = cross_validate(M,
                               X,
                               y,
                               cv = cross_val)
    # save cross validation error into result vector
    cv_error[i] = np.mean(cv_results['test_score'])
cv_error

# Part 2: Case study cross validation (no solutions beyond this point as we did not get further in lecture)

(see Exercise 5.4.5)

In this case study we use the credit card dataset to predict the probability of default. We will build a logistic regression model and estimate its test error using the validation set approach and the cross-validation approach.

In [ ]:
# run this cell to load the data
Default = load_data('Default')
Default

Background information on the dataset can be found [in the documentation](https://islp.readthedocs.io/en/latest/datasets/Default.html).

## Task 2.1
Fit a logistic regression model that uses `income` and `balance` to predict `default`.

In [ ]:
# your code here
...

## Task 2.2
Using the validation set approach, estimate the test error of this model. In order to do this, you must perform the following steps:

i. Split the sample set into a training set and a validation set.

ii. Fit a multiple logistic regression model using only the training observations.

iii. Obtain a prediction of default status for each individual in the validation set by computing the posterior probability of default for that individual, and classifying the individual to the default category if the posterior probability is greater than 0.5.

iv. Compute the validation set error, which is the fraction of the observations in the validation set that are misclassified.

In [ ]:
# your code here
...

## Task 2.3
Repeat the process in (b) three times, using three different splits of the observations into a training set and a validation set. Comment on the results obtained.

In [ ]:
# your code here
...

## Task 2.4
Now predict the test error of the model using 10-fold cross-validation. To do so, follow the steps we developed in Part 1b) of this notebook.

**Note**: To carry out this task, we need to define our own custom score using [`sklearn.make_scorer()`](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.make_scorer.html) and pass this to the function [`sklearn.cross_validate()`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html) by specifying it as an argument to the `scoring` parameter.

Our custom scoring function needs to have the signature `score_func(y, y_pred, **kwargs)` with `y` being the true labels and `y_pred` the predicted labels as output by `sm.Logit()` when applying the `predict()` method. Since `statsmodels` outputs probabilities rather than actual labels, we first transform these probabilities  into labels. This is what our scoring function `accuracy_score_sm()` does.

In [ ]:
# Step 1: Define custom scorer which computes accuracy based on probabilities as
# output by statsmodels
from sklearn.metrics import make_scorer
def accuracy_score_sm(y_true, y_pred_prob):
    # computes the accuracy for the output of a binary classification model

    # inputs:
    #    y_true: ground truth labels, encoded as 0,1
    #    y_pred_prob: probabilities for label 1 as predicted by the model

    # output:
    #    percentage of models 
    y_pred = ...
    return ...
accuracy_sm = make_scorer(...)

# Step 2: Initialize splitter for cross validation and model
cross_val = ...
M = sklearn_sm(sm.Logit)

# Step 3: Define response variable and design matrix for cross validation
y = ...
predictors = ...
X = ...

# Step 4: Run cross validation
cv_results = cross_validate(M,
                            X,
                            y,
                            scoring = accuracy_sm,
                            cv = cross_val)
print('Mean accuracy: ', np.mean(cv_results['test_score']))

## Task 2.5
Now consider a logistic regression model that predicts the probability of default using income, balance, and a dummy variable for student. Estimate the test error for this model using 10-fold cross-validation. Comment on whether or not including a dummy variable for student leads to a reduction in the test error rate.

In [ ]:
# your code here
...